In [3]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import requests
from Bio import Entrez
from pathlib import Path
from tenacity import retry, stop_after_attempt, wait_exponential
import ttkbootstrap as ttk  # Modern theme for tkinter

# Configure Entrez (NCBI)
Entrez.email = "your_email@example.com"  # Required by NCBI
NCBI_DELAY = 0.34  # Respect NCBI's 3 requests/second limit

class AccessionDownloaderGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Accession Number Downloader")
        self.root.geometry("600x400")
        self.style = ttk.Style(theme="flatly")  # Choose a theme: 'cosmo', 'flatly', etc.

        # Variables
        self.source_var = tk.StringVar(value="uniprot")
        self.format_var = tk.StringVar(value="fasta")
        self.input_file = tk.StringVar()
        self.output_dir = tk.StringVar(value="downloads")
        self.log_text = tk.StringVar(value="Ready to download...\n")

        # GUI Layout
        self.setup_ui()

    def setup_ui(self):
        # Frame for input controls
        control_frame = ttk.Frame(self.root, padding="10")
        control_frame.pack(fill=tk.X)

        # Database Selection
        ttk.Label(control_frame, text="Data Source:").grid(row=0, column=0, sticky=tk.W)
        sources = [("UniProt", "uniprot"), ("NCBI", "ncbi"), ("Ensembl", "ensembl")]
        for i, (text, val) in enumerate(sources):
            ttk.Radiobutton(control_frame, text=text, variable=self.source_var, value=val).grid(row=0, column=i+1, sticky=tk.W)

        # Format Selection (UniProt only)
        ttk.Label(control_frame, text="Format:").grid(row=1, column=0, sticky=tk.W)
        formats = [("FASTA", "fasta"), ("JSON", "json")]
        for i, (text, val) in enumerate(formats):
            rb = ttk.Radiobutton(control_frame, text=text, variable=self.format_var, value=val)
            rb.grid(row=1, column=i+1, sticky=tk.W)
            # Disable format selection if source is not UniProt
            if self.source_var.get() != "uniprot":
                rb.state(["disabled"])

        # Input File Selection
        ttk.Label(control_frame, text="Accession File:").grid(row=2, column=0, sticky=tk.W)
        ttk.Entry(control_frame, textvariable=self.input_file, width=40).grid(row=2, column=1, columnspan=2, sticky=tk.EW)
        ttk.Button(control_frame, text="Browse", command=self.browse_file).grid(row=2, column=3, sticky=tk.E)

        # Output Directory
        ttk.Label(control_frame, text="Output Folder:").grid(row=3, column=0, sticky=tk.W)
        ttk.Entry(control_frame, textvariable=self.output_dir, width=40).grid(row=3, column=1, columnspan=2, sticky=tk.EW)
        ttk.Button(control_frame, text="Browse", command=self.browse_output).grid(row=3, column=3, sticky=tk.E)

        # Download Button
        ttk.Button(control_frame, text="Download", command=self.start_download, bootstyle="success").grid(row=4, column=0, columnspan=4, pady=10)

        # Log Output
        log_frame = ttk.Frame(self.root, padding="10")
        log_frame.pack(fill=tk.BOTH, expand=True)
        ttk.Label(log_frame, text="Download Log:").pack(anchor=tk.W)
        self.log_area = tk.Text(log_frame, wrap=tk.WORD, height=10)
        self.log_area.pack(fill=tk.BOTH, expand=True)
        scrollbar = ttk.Scrollbar(log_frame, command=self.log_area.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_area.config(yscrollcommand=scrollbar.set)

        # Bind source change to update format radio buttons
        self.source_var.trace_add("write", self.update_format_options)

    def browse_file(self):
        filepath = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")])
        if filepath:
            self.input_file.set(filepath)

    def browse_output(self):
        dirpath = filedialog.askdirectory()
        if dirpath:
            self.output_dir.set(dirpath)

    def update_format_options(self, *args):
        # Enable/disable format options based on source
        for widget in self.root.winfo_children():
            if isinstance(widget, ttk.Radiobutton) and "json" in widget.cget("value"):
                if self.source_var.get() == "uniprot":
                    widget.state(["!disabled"])
                else:
                    widget.state(["disabled"])

    def log(self, message):
        self.log_area.insert(tk.END, message + "\n")
        self.log_area.see(tk.END)
        self.root.update_idletasks()

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def _fetch_url(self, url, headers=None):
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response

    def download_uniprot(self, accession, output_dir):
        """Download UniProt entry in selected format."""
        url = f"https://www.uniprot.org/uniprotkb/{accession}.{self.format_var.get()}"
        try:
            response = self._fetch_url(url)
            ext = self.format_var.get()
            out_file = Path(output_dir) / f"{accession}.{ext}"
            with open(out_file, "w") as f:
                f.write(response.text)
            self.log(f"✅ UniProt: Saved {accession}.{ext}")
        except Exception as e:
            self.log(f"❌ UniProt: Failed {accession} ({e})")

    def download_ncbi(self, accession, output_dir):
        """Fetch NCBI data (protein/nucleotide) in FASTA."""
        try:
            handle = Entrez.efetch(db="protein", id=accession, rettype="fasta", retmode="text")
            data = handle.read()
            out_file = Path(output_dir) / f"{accession}.fasta"
            with open(out_file, "w") as f:
                f.write(data)
            self.log(f"✅ NCBI: Saved {accession}.fasta")
            time.sleep(NCBI_DELAY)  # Rate limiting
        except Exception as e:
            self.log(f"❌ NCBI: Failed {accession} ({e})")

    def download_ensembl(self, accession, output_dir):
        """Fetch Ensembl gene/transcript sequence."""
        url = f"https://rest.ensembl.org/sequence/id/{accession}?content-type=text/plain"
        try:
            response = self._fetch_url(url)
            out_file = Path(output_dir) / f"{accession}.fasta"
            with open(out_file, "w") as f:
                f.write(f">{accession}\n{response.text}")
            self.log(f"✅ Ensembl: Saved {accession}.fasta")
        except Exception as e:
            self.log(f"❌ Ensembl: Failed {accession} ({e})")

    def start_download(self):
        """Main download function triggered by the GUI button."""
        if not self.input_file.get():
            messagebox.showerror("Error", "Please select an input file!")
            return

        # Read accessions from file
        try:
            with open(self.input_file.get(), "r") as f:
                accessions = [line.strip() for line in f if line.strip()]
        except Exception as e:
            messagebox.showerror("Error", f"Failed to read input file: {e}")
            return

        # Create output directory
        output_dir = Path(self.output_dir.get())
        output_dir.mkdir(exist_ok=True)

        # Start downloads
        self.log("\n=== Starting Downloads ===")
        source = self.source_var.get()
        for acc in accessions:
            if source == "uniprot":
                self.download_uniprot(acc, output_dir)
            elif source == "ncbi":
                self.download_ncbi(acc, output_dir)
            elif source == "ensembl":
                self.download_ensembl(acc, output_dir)
        self.log("=== Download Complete ===")

if __name__ == "__main__":
    root = ttk.Window()
    app = AccessionDownloaderGUI(root)
    root.mainloop()

ModuleNotFoundError: No module named 'Bio'